In [1]:
import logging
import sys
import time
from diskchef.model.model import Model
from pathlib import Path
import astropy.table
from diskchef.lamda.line import Line
from diskchef.physics.yorke_bodenheimer import YorkeBodenheimer2008
from diskchef.uv.uvfits_to_visibilities_ascii import UVFits
from astropy import units as u
import numpy as np
import emcee
from multiprocessing import Pool
import tempfile
import spectral_cube
from matplotlib import pyplot as plt
import ipywidgets

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                    level=logging.INFO, stream=sys.stdout)

logging.info("Starting demo")

2021-07-05 10:37:50,927 | INFO : Starting demo


In [3]:
lines = [
    # Line(name='HCN J=3-2', transition=3, molecule='HCN'),
    # Line(name='HCO+ J=3-2', transition=3, molecule='HCO+'),
    # Line(name='N2H+ J=3-2', transition=3, molecule='N2H+'),
    Line(name='CO J=2-1', transition=2, molecule='CO'),
]

yb = YorkeBodenheimer2008()

2021-07-02 16:24:20,501 | INFO : Creating an instance of Line
2021-07-02 16:24:20,507 | INFO : Found LAMDA file: /mnt/d/astrowork/diskchef/diskchef/lamda/files/co.dat


## Play with different models

In [4]:
def plot_model(mass, gas_mass, bins, temperature_range_1au,
               tapering_radius, tapering_gamma,
               temperature_slope,
               mmw, inner_radius,
               rminmax):
    mass = mass * u.M_sun
    gas_mass = gas_mass * u.M_sun
    print(f"mass: {mass}")
    print(f"gas mass: {gas_mass}")
    logging.info(f"mass: {mass}\ngas mass: {gas_mass}")
    folder = Path(tempfile.mkdtemp(prefix="fit_", dir="play"))
    try:
        demo_model = Model(
            disk="Fit",
            line_list=lines,
            params=dict(
                r_min=rminmax[0] * u.au, r_max=rminmax[1] * u.au, radial_bins=bins, vertical_bins=bins,
                gas_mass=gas_mass,
                midplane_temperature_1au=temperature_range_1au[0] * u.K,
                atmosphere_temperature_1au=temperature_range_1au[1] * u.K,
                tapering_radius=tapering_radius * u.au,
                tapering_gamma=tapering_gamma,
                inner_radius=inner_radius * u.au,
                mean_molecular_mass=mmw * u.g / u.mol,
                temperature_slope=temperature_slope,
            ),
            rstar=yb.radius(mass),
            tstar=yb.effective_temperature(mass),
            inc=0 * u.deg,
            PA=0 * u.deg,
            distance=0 * u.pc,
            # nphot_therm=3e5,
            npix=120,
            channels=21,
            folder=folder,
            run_mctherm=False,
            run=False
        )
        demo_model.chemistry()
        fig = demo_model.plot(save=False)
    except Exception as e:
        return e

In [5]:
ipywidgets.interact_manual(
    plot_model,
    mass=ipywidgets.FloatLogSlider(min=-1, max=1, base=10, value=1),
    gas_mass=ipywidgets.FloatLogSlider(min=-3, max=0, base=10, value=1e-2),
    bins=ipywidgets.IntSlider(min=5, max=100, value=30),
    temperature_range_1au=ipywidgets.FloatRangeSlider(min=50, max=1100, step=10,
                                                      value=[200, 1000], readout_format="d"),
    rminmax=ipywidgets.FloatRangeSlider(min=0.01, max=1000, step=10, value=[1, 400]),
    tapering_radius=ipywidgets.IntSlider(min=10, max=1000, value=100),
    tapering_gamma=ipywidgets.FloatSlider(min=0.1, max=2., value=0.75),
    temperature_slope=ipywidgets.FloatSlider(min=0.1, max=2., value=0.55),
    mmw=ipywidgets.FloatSlider(min=1, max=30, value=2.33),
    inner_radius=ipywidgets.FloatSlider(min=0.1, max=10, value=1)
)


interactive(children=(FloatLogSlider(value=1.0, description='mass', max=1.0, min=-1.0), FloatLogSlider(value=0…

<function __main__.plot_model(mass, gas_mass, bins, temperature_range_1au, tapering_radius, tapering_gamma, temperature_slope, mmw, inner_radius, rminmax)>